# Import Libs

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers 



import os,warnings,csv,shutil
import cv2
import math

from tensorflow.keras.models import model_from_json

# Load Name of files and your labels

In [ ]:
X_train = np.load('/kaggle/input/birdclef-2020/X_train.npy')
y_train = np.load('/kaggle/input/birdclef-2020/y_train.npy')

X_val = np.load('/kaggle/input/birdclef-2020/X_val.npy')
y_val = np.load('/kaggle/input/birdclef-2020/y_val.npy')

# Load Models

In [ ]:
# load json and create model
json_file = open('../input/validate/MultiScale-Triplet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
multiscale = model_from_json(loaded_model_json)
# load weights into new model
multiscale.load_weights("../input/validate/model00000035.h5")
print("Loaded model from disk")

# Make a DataGenerator to get the numpy arrays

In [ ]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, image_filenames,labels, image_path,to_fit=True, batch_size=64):
        self.labels = labels
        self.image_filenames = image_filenames
        self.image_path = image_path
        self.to_fit = to_fit
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        # Generate data
        X = self._generate_X(index)

        if self.to_fit:
            y = self._generate_y(index)
            return X, y
        else:
            return X

    def _generate_X(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        ret = []

        for file_name in batch_x:
            dict_data = np.load(self.image_path +str(file_name)+".npz")
            data = dict_data['arr_0']
            ret.append(data)

        return np.array(ret)

    def _generate_y(self, idx):
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array(batch_y)

# Get Data to Classify

In [ ]:
image_path = '../input/birdclef-2020/train/train/'
pred_generator = DataGenerator(X_train, y_train, image_path, to_fit=False,batch_size=1)
pred_val_generator = DataGenerator(X_val, y_val, image_path, to_fit=False,batch_size=1)

In [ ]:
#Train embeddings
pred = multiscale.predict(pred_generator)
pred.shape

In [ ]:
#Validation embeddings
pred_val = multiscale.predict(pred_val_generator)
pred_val.shape

In [ ]:
# Transform y_train and y_val to onehot way
emb_train = pred
y_train_onehot = keras.utils.to_categorical(y_train)
emb_val = pred_val
y_val_onehot = keras.utils.to_categorical(y_val)

## MPL Classifier

In [ ]:
#Input Layer
Input = keras.Input(shape=(128))
#Hidden Layer
x = layers.Dense(units=256,activation='relu')(Input)
x = layers.Dropout(0.5)(x)
#Output to N Classes
x = layers.Dense(units=960,activation='softmax')(x)

mlp = keras.Model(inputs=Input, outputs=x, name="MLP")

#To see the summary of model uncoment here
#model2.summary()
#To see a plot of model uncoment here
#keras.utils.plot_model(model2, "classification.png", show_shapes=True)

In [ ]:
mlp.compile(
    optimizer=keras.optimizers.Adam(0.001), 
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
    )

In [ ]:
history = mlp.fit(emb_train, y_train_onehot, verbose=1,validation_data=(emb_val,y_val_onehot),batch_size=256,epochs=200)

In [ ]:
model_json = mlp.to_json()
with open("Classifier.json", "w") as json_file:
    json_file.write(model_json)
mlp.save_weights("classifier.h5")    
print("Saved model to disk")

In [ ]:
#Plot the train things
plt.plot(history.history['loss'][:])
plt.plot(history.history['val_loss'][:])
plt.title('model loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Classify and Get accuracy from 2 Classifiers

In [ ]:
#Get The 2 Fist Classes
def my_predict(y_in_perc):
    n_pred = [[],[]]
    n_pred[0] = np.zeros(len(y_in_perc),dtype=int)
    n_pred[1] = np.zeros(len(y_in_perc),dtype=int)
    
    for i in range(len(y_in_perc)):  
        #if(y_in_perc[i].max() > 0.3):
        ma1 = 0
        ma2 = 0
        
        for j in range(y_in_perc[i].shape[0]):
            if(y_in_perc[i][j] > y_in_perc[i][ma1]):
                ma2 = ma1
                ma1 = j
            elif (y_in_perc[i][j] > y_in_perc[i][ma2]):
                ma2 = j
                
                
        n_pred[0][i] = ma1
        n_pred[1][i] = ma2
        
    return n_pred 

In [ ]:
def Evaluate(y_train_pred,y_val_pred):  
    # calculate training accuracy
    correct_preds = 0
    for i in range(len(y_train)):
        if (y_train[i] == y_train_pred[0][i] ):
            correct_preds  += 1
    train_acc = correct_preds / y_train.shape[0]

    print(f'Training accuracy: {(train_acc * 100):.2f}')

    # calculate testing accuracy
    correct_preds = 0
    for i in range(len(y_val)):
        if(y_val[i] == y_val_pred[0][i]):
            correct_preds += 1
    test_acc = correct_preds / y_val.shape[0]

    print(f'Test accuracy: {(test_acc * 100):.2f}')

In [ ]:
#Predict from MLP
y_train_pred = mlp.predict(emb_train)
y_val_pred = mlp.predict(emb_val)

#Get the 2 first
y_train_pred = my_predict(y_train_pred)
y_val_pred = my_predict(y_val_pred)

Evaluate(y_train_pred,y_val_pred)

# Knn Classifier

In [ ]:
feature_number = len(emb_train[0])

k = 7

In [ ]:
x_data_train = tf.Variable(emb_train)
y_data_train = tf.Variable(y_train_onehot)


accuracy = 0

for i in range(0,(len(emb_val)//10)):    
    x_data_test = tf.Variable(emb_val[i*10:(i+1)*10])

    # manhattan distance
    distance = tf.reduce_sum(tf.abs(tf.subtract(x_data_train, tf.expand_dims(x_data_test, 1))), axis=2)

    # nearest k points
    _, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
    top_k_label = tf.gather(y_data_train, top_k_indices)

    sum_up_predictions = tf.reduce_sum(top_k_label, axis=1)
    prediction = tf.argmax(sum_up_predictions, axis=1)

    for pred, actual in zip(prediction, y_val_onehot[i*10:(i+1)*10]):
        if pred == np.argmax(actual):
            accuracy += 1

            
print(accuracy)
print(accuracy / len(emb_val))